# LangChain 实战：汽车销售聊天机器人

## 使用 GPT-4 构造销售话术数据



使用 ChatGPT 构造销售数据的 Prompt 示例：

```
你是汽车金牌销售员，现在培训职场新人，请给出100条实用的销售话术。

每条销售话术以如下格式给出：
[客户问题]
[销售回答]

```

GPT-4 回复结果：

```
当然，下面是100条实用的汽车销售话术，按照提供的格式列出：

1. [客户问题] 我不确定这款车是否适合我的需求。
   [销售回答] 您的需求是什么？我们可以一起看看这款车是否能够满足您的所有要求。

2. [客户问题] 我觉得这款车的价格有点高。
   [销售回答] 我理解您的顾虑。我们可以看看是否有适合您预算的其他选项，或者我们也有一些灵活的融资方案可以帮助您。

3. [客户问题] 这款车的油耗如何？
   [销售回答] 这款车的油耗非常经济，每百公里仅需X升，非常适合长途驾驶和日常通勤。

4. [客户问题] 这款车的安全性能怎么样？
   [销售回答] 这款车配备了多项先进的安全技术，例如自适应巡航控制、车道保持辅助和自动紧急制动系统，确保您的安全。

5. [客户问题] 这款车有多少保修期？
   [销售回答] 这款车提供5年或10万公里的保修期，此外我们还有延长保修服务可以选择。

6. [客户问题] 这款车的售后服务怎么样？
   [销售回答] 我们的售后服务非常全面，包括定期保养、紧急救援和免费的软件更新，确保您无忧驾驶。

7. [客户问题] 我能试驾一下这款车吗？
   [销售回答] 当然可以！我现在就为您安排试驾，让您亲自体验这款车的驾驶感受。

8. [客户问题] 这款车的保养费用贵吗？
   [销售回答] 这款车的保养费用非常合理，并且我们有多种保养套餐供您选择，您可以根据自己的需求来选择最适合的套餐。

9. [客户问题] 这款车的保险费用高吗？
   [销售回答] 保险费用取决于多种因素，包括您的驾驶记录和选择的保险公司。我们可以为您提供多种报价，帮您找到最合适的方案。

10. [客户问题] 这款车有折扣吗？
    [销售回答] 目前我们有一些促销活动和特别折扣，我可以为您详细介绍一下，帮您享受最大的优惠。
```


## 使用 Document Transformers 模块来处理原始数据


将 ChatGPT 生成的结果保存到 [real_car_sales_data.txt](real_car_sales_data.txt) 文件中

In [2]:
with open("real_car_sales_data.txt", encoding='utf-8') as f:
    real_car_sales = f.read()

### 使用 CharacterTextSplitter 来进行文本分割

- 基于单字符来进行文本分割（separator）
- 基于字符数来决定文本块长度（chunk_size）

参考示例：

```python
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)
```


In [3]:
from langchain.text_splitter import CharacterTextSplitter

In [4]:
text_splitter = CharacterTextSplitter(        
    separator = r'\d+\.',
    chunk_size = 100,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [5]:
docs = text_splitter.create_documents([real_car_sales])

In [6]:
docs[0]

Document(page_content='[客户问题] 我不确定这款车是否适合我的需求。\n   [销售回答] 您的需求是什么？我们可以一起看看这款车是否能够满足您的所有要求。')

In [7]:
len(docs)

100

### 使用 Faiss 作为向量数据库，持久化存储汽车销售 问答对（QA-Pair）

In [9]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, OpenAIEmbeddings(base_url="https://api.xiaoai.plus/v1",api_key="sk-YIOnMFnuWMDaLG0bC93a20F2456b4dAdAa4aE692449483E7"))

In [13]:
query = "这台车耗油吗"

In [14]:
answer_list = db.similarity_search(query)

In [15]:
for ans in answer_list:
    print(ans.page_content + "\n")

[客户问题] 这款车的油耗如何？
   [销售回答] 这款车的油耗非常经济，每百公里仅需X升，非常适合长途驾驶和日常通勤。

[客户问题] 这款车的油耗是多少？
    [销售回答] 这款车的油耗非常经济，每百公里仅需X升，非常适合长途驾驶和日常通勤。

[客户问题] 这款车有自动启停功能吗？
    [销售回答] 是的，这款车配备了自动启停功能，可以在红灯和交通

堵塞时自动关闭发动机，节省燃油。

[客户问题] 这款车的尾气排放标准是什么？
    [销售回答] 这款车符合最新的尾气排放标准，环保性能非常好，符合国家的环保政策。



In [17]:
db.save_local("real_car_sale")

### 使用 retriever 从向量数据库中获取结果

#### 使用参数 `k` 指定返回结果数量


In [18]:
# 实例化一个 TopK Retriever
topK_retriever = db.as_retriever(search_kwargs={"k": 3})

In [19]:
topK_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000029CE8327CD0>, search_kwargs={'k': 3})

In [21]:
docs = topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 这款车的油耗如何？
   [销售回答] 这款车的油耗非常经济，每百公里仅需X升，非常适合长途驾驶和日常通勤。

[客户问题] 这款车的油耗是多少？
    [销售回答] 这款车的油耗非常经济，每百公里仅需X升，非常适合长途驾驶和日常通勤。

[客户问题] 这款车有自动启停功能吗？
    [销售回答] 是的，这款车配备了自动启停功能，可以在红灯和交通

堵塞时自动关闭发动机，节省燃油。



In [115]:
docs = topK_retriever.get_relevant_documents("孩子上学方便吗")

In [22]:
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 这款车的油耗如何？
   [销售回答] 这款车的油耗非常经济，每百公里仅需X升，非常适合长途驾驶和日常通勤。

[客户问题] 这款车的油耗是多少？
    [销售回答] 这款车的油耗非常经济，每百公里仅需X升，非常适合长途驾驶和日常通勤。

[客户问题] 这款车有自动启停功能吗？
    [销售回答] 是的，这款车配备了自动启停功能，可以在红灯和交通

堵塞时自动关闭发动机，节省燃油。



#### 使用 similarity_score_threshold 设置阈值，提升结果的相关性质量

In [23]:
# 实例化一个 similarity_score_threshold Retriever
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.8}
)

In [24]:
docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 这款车的油耗如何？
   [销售回答] 这款车的油耗非常经济，每百公里仅需X升，非常适合长途驾驶和日常通勤。

[客户问题] 这款车的油耗是多少？
    [销售回答] 这款车的油耗非常经济，每百公里仅需X升，非常适合长途驾驶和日常通勤。



### 提取向量数据库中的`销售回答`

In [25]:
docs = retriever.get_relevant_documents(query)

In [26]:
docs[0].page_content

'[客户问题] 这款车的油耗如何？\n   [销售回答] 这款车的油耗非常经济，每百公里仅需X升，非常适合长途驾驶和日常通勤。'

In [27]:
docs[0].page_content.split("[销售回答] ")

['[客户问题] 这款车的油耗如何？\n   ', '这款车的油耗非常经济，每百公里仅需X升，非常适合长途驾驶和日常通勤。']

In [28]:
ans = docs[0].page_content.split("[销售回答] ")[-1]

In [29]:
ans

'这款车的油耗非常经济，每百公里仅需X升，非常适合长途驾驶和日常通勤。'

#### 尝试各种问题

In [30]:
from typing import List

def sales(query: str, score_threshold: float=0.8) -> List[str]:
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})    
    docs = retriever.get_relevant_documents(query)
    ans_list = [doc.page_content.split("[销售回答] ")[-1] for doc in docs]
    return ans_list

In [31]:
query = "这个价格太高了吧"

print(sales(query))

[]


D:\tools\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


In [33]:
print(sales(query, 0.7))

['我理解您的顾虑。我们可以看看是否有适合您预算的其他选项，或者我们也有一些灵活的融资方案可以帮助您。', '这款车的维护费用非常合理，并且我们提供多种维护套餐，可以根据您的需求选择最适合的方案。', '这款车的保养费用非常合理，并且我们有多种保养套餐供您选择，您可以根据自己的需求来选择最适合的套餐。', '保险费用取决于多种因素，包括您的驾驶记录和选择的保险公司。我们可以为您提供多种报价，帮您找到最合适的方案。']


In [35]:
query = "舒适度要好一点"

print(f"score:0.8 ans: {sales(query)}\n")
print(f"score:0.75 ans: {sales(query, 0.75)}\n")
print(f"score:0.5 ans: {sales(query, 0.5)}\n")

D:\tools\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


score:0.8 ans: []

score:0.75 ans: ['这款车的座椅采用人体工程学设计，并且有多向电动调节功能，坐起来非常舒适。']

score:0.5 ans: ['这款车的座椅采用人体工程学设计，并且有多向电动调节功能，坐起来非常舒适。', '这款车的座椅通风功能非常好，可以在炎热的夏天提供凉爽的座椅体验，让您更加舒适。', '是的，这款车的座椅支持多向电动调节，可以轻松找到最舒适的坐姿。', '这款车的座椅加热功能非常好，可以快速加热，让您在寒冷的冬\n\n天也能享受温暖的驾驶体验。']



#### 当向量数据库中没有合适答案时，使用大语言模型能力

In [36]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0.5, base_url="https://api.xiaoai.plus/v1",api_key="sk-YIOnMFnuWMDaLG0bC93a20F2456b4dAdAa4aE692449483E7")
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [38]:
qa_chain({"query": "汽车保养费高吗"})

{'query': '汽车保养费高吗',
 'result': '这款车的保养费用非常合理，并且我们有多种保养套餐供您选择，您可以根据自己的需求来选择最适合的套餐。'}

In [40]:
qa_chain({"query": "最便宜的汽车配置需要多少钱"})

D:\tools\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '最便宜的汽车配置需要多少钱',
 'result': '对不起，我无法提供具体的最便宜的汽车配置的价格。汽车价格会因品牌、型号、地区、税收、经销商定价策略以及购车时的各种优惠等因素而有很大的差异。通常，各个汽车制造商都会提供不同配置的车型，包括入门级（通常是最便宜的）以及配备更多功能的高级版本。\n\n如果你想了解特定市场中最便宜的汽车配置的价格，最好的方法是直接访问汽车制造商的官方网站，或联系当地的汽车经销商进行询价。此外，价格信息也可能在汽车评测网站、汽车购买指南或者在线汽车市场上找到。请注意，最便宜的配置可能会缺少某些舒适性或安全性功能。'}

In [41]:
print(sales("最便宜的汽车配置需要多少钱"))

[]


D:\tools\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


## 加载 FAISS 向量数据库已有结果

In [44]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

db = FAISS.load_local("real_car_sale", OpenAIEmbeddings(base_url="https://api.xiaoai.plus/v1",api_key="sk-YIOnMFnuWMDaLG0bC93a20F2456b4dAdAa4aE692449483E7"), allow_dangerous_deserialization='True')

In [46]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4", temperature=0.5, base_url="https://api.xiaoai.plus/v1",api_key="sk-YIOnMFnuWMDaLG0bC93a20F2456b4dAdAa4aE692449483E7")
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [47]:
qa_chain({"query": "那种车比较适合年轻人"})

D:\tools\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '那种车比较适合年轻人',
 'result': '适合年轻人的车型因个人喜好、预算和需求而异。一般来说，年轻人可能更喜欢外观时尚、操控性能好、油耗低的车型。一些热门的选择包括：本田飞度（Honda Fit）、大众高尔夫（Volkswagen Golf）、福特嘉年华（Ford Fiesta）和马自达3（Mazda3）。最终选择还需根据个人实际情况来决定。'}

In [48]:
# 输出内部 Chain 的日志
qa_chain.combine_documents_chain.verbose = True

In [49]:
qa_chain({"query": "那种车比较适合年轻人"})

D:\tools\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(




> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': '那种车比较适合年轻人',
 'result': '对不起，您提供的信息不足以给出一个准确的答案。年轻人选择车辆时，可能会考虑很多因素，比如价格、燃油经济性、车辆性能、设计、安全性等。最好的建议是，年轻人应该根据自己的预算、需求和喜好来选择车辆。'}

In [50]:
# 返回向量数据库的检索结果
qa_chain.return_source_documents = True

In [53]:
result = qa_chain({"query": "那种车比较适合年轻人"})

D:\tools\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(




> Entering new StuffDocumentsChain chain...

> Finished chain.


In [54]:
result

{'query': '那种车比较适合年轻人',
 'result': '这个问题的答案可能会因人而异，因为不同的年轻人可能有不同的需求和喜好。但一般来说，一些小型车和紧凑型车，如本田Fit，福特Fiesta，或丰田Yaris可能比较适合年轻人，因为它们经济实用，容易驾驶和停车。此外，一些有科技感和环保意识的年轻人可能会选择电动车，如特斯拉Model 3。但最重要的是根据个人的需求和预算来选择车辆。',
 'source_documents': []}